## Image feature extraction

In [1]:
from util import *

In [2]:
IMAGE_MODEL = 'inception'

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

cuda:0
Num GPUs Available:  1


In [4]:
from mlflow import log_metric, log_param, log_artifact
import mlflow.sklearn
import mlflow.xgboost


from mlflow.tracking import MlflowClient


In [5]:
remote_server_uri = "http://127.0.0.1:5000/"  # set to your server URI
mlflow.set_tracking_uri(remote_server_uri)
experiment_name = 'Men_Clothing_Image_recommendation_v4'

mlflow.set_experiment(experiment_name)
# Forcing an end_run() to prevent 
#    https://github.com/mlflow/mlflow/issues/1335 
#    https://github.com/mlflow/mlflow/issues/608

mlflow.autolog()

artifact_path = mlflow.get_artifact_uri()
uri = mlflow.tracking.get_tracking_uri()
print(artifact_path)
print(uri)
mlflow.end_run()

2025/02/22 11:40:43 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2025/02/22 11:40:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/02/22 11:40:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for tensorflow.
2025/02/22 11:40:43 INFO mlflow.tracking.fluent: Autologging successfully enabled for keras.
2025/02/22 11:40:43 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This ini

mlflow-artifacts:/837799944280473961/5690c2ea4d0841cda482d1b25c7f7a68/artifacts
http://127.0.0.1:5000/


In [6]:
from mlflow.tracking import MlflowClient
client = MlflowClient(mlflow.get_tracking_uri())


In [7]:
nodes = pd.read_parquet(f'../Dataset/nodes_without_image.parquet')
# edges = pd.read_parquet(f'../processed/{IMAGE_MODEL}/edges.parquet',engine='pyarrow')
umap_dt = {'n_components':10}

In [8]:
# g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g,train_pos_g,test_pos_g,train_neg_g,test_neg_g,cols_needed = create_model_data(IMAGE_MODEL,True,0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3})

## With Only edges information

In [9]:
cols_needed = ['asin', 'price_power_transform', 'also_buy', 'avg_rating',
       'no_of_ratings', 'rank_num_power_transform', 'label_1', 'label_2',
       'label_3', 'label_4', 'label_5', 'label_6', 'label_7', 'label_8',
       'label_9', 'label_10', 'label_11', 'label_12', 'label_13',
       'cluster_group_cluster 0', 'cluster_group_cluster 1',
       'cluster_group_cluster 2', 'cluster_group_cluster 3',
       'cluster_group_cluster 4', 'cluster_group_cluster 5',
       'cluster_group_cluster 6', 'cluster_group_cluster 7',
       'cluster_group_cluster 8', 'cluster_group_cluster 9',
       'clothing_type_Active', 'clothing_type_Jackets & Coats',
       'clothing_type_Jeans', 'clothing_type_Pants', 'clothing_type_Shirts',
       'clothing_type_Shorts', 'clothing_type_Sleep & Lounge',
       'clothing_type_Socks', 'clothing_type_Swim', 'clothing_type_Underwear']

In [10]:
# for i in nodes.columns:
#     nodes[i] = 1

In [12]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'mean', 'dropout': 0, 'h_feats': 16, 'lr': 0.01},#{'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_same_values_ndata',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=True,use_bidirectional=True)

TypeError: create_model_data() got an unexpected keyword argument 'cols_needed'

In [18]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'mean', 'dropout': 0, 'h_feats': 16, 'lr': 0.01},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_price_feature',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=True,use_bidirectional=False,cols_needed=['price_power_transform'])

Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 1
node features shape : torch.Size([10867, 1])
14388 6166
In epoch 0, loss: 1.420879602432251
Epoch 0: Best model saved min loss 1.420879602432251
Epoch 1: Best model saved min loss 1.2001255750656128
Epoch 2: Best model saved min loss 1.0127512216567993
Epoch 3: Best model saved min loss 0.8594118356704712
Epoch 4: Best model saved min loss 0.743012547492981
In epoch 5, loss: 0.6625522971153259
Epoch 5: Best model saved min loss 0.6625522971153259
Epoch 6: Best model saved min loss 0.6057570576667786
Epoch 7: Best model saved min loss 0.5590574145317078
Epoch 8: Best model saved min loss 0.5177666544914246
Epoch 9: Best model saved min loss 0.4829390347003937
In epoch 10, loss: 0.45662063360214233
Epoch 10: Best model saved min loss 0.45662063360214233
Epoch 11: Best model saved min loss 0.4390198588371277
Epoch 12: Best model saved min loss 0.42797043919563293
Epoch 13: Best model saved min loss 0.42

100%|██████████| 10867/10867 [00:01<00:00, 8693.56it/s]


Train recall@100 0.24386655199397875
Test recall@100 0.2679001903150772


In [16]:
g.ndata['feat']

tensor([[ 1.2678],
        [ 1.0752],
        [ 1.0752],
        ...,
        [-0.8801],
        [ 0.8022],
        [-0.4471]])

In [15]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_no_features',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=False,skip_ndata=True,use_bidirectional=True)

Node data shape : (10867, 39)
Edges data shape : (20554, 2)
Size of Input features : 37
node features shape : torch.Size([10867, 37])
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 12.481778144836426
Epoch 0: Best model saved min loss 12.481778144836426
Epoch 1: Best model saved min loss 4.026902675628662
Epoch 2: Best model saved min loss 0.9980332851409912
Epoch 3: Best model saved min loss 0.5205588340759277
Epoch 4: Best model saved min loss 0.4308769404888153
In epoch 5, loss: 0.4118104875087738
Epoch 5: Best model saved min loss 0.4118104875087738
Epoch 7: Best model saved min loss 0.40738698840141296
Epoch 8: Best model saved min loss 0.3867255449295044
Epoch 9: Best model saved min loss 0.3637147843837738
In epoch 10, loss: 0.347954124212265
Epoch 10: Best model saved min loss 0.347954124212265
Epoch 11: Best model saved min loss 0.33972927927970886
Epoch 12: Best model saved min loss 0.33568254113197327
Epoc

100%|██████████| 10867/10867 [00:01<00:00, 7879.26it/s]


Train recall@100 0.6104862324571897
Test recall@100 0.5978765462443362


In [16]:
g.edges()

(tensor([    0,     1,     1,  ..., 10864, 10865, 10866]),
 tensor([7018, 2719, 5903,  ..., 8566, 3451, 8679]))

In [ ]:
# IMAGE_MODEL = 'inception'
# temp = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
#                                                                                                                                           run_name=f'{IMAGE_MODEL}_best_tuned_model',
#                                                                                                                                           n_epochs=1000,
#                                                                                                                                           image_included=False,
#                                                                                                                                           split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100)

## Without using any image data

In [9]:
IMAGE_MODEL = 'inception'
temp = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_best_tuned_model',
                                                                                                                                          n_epochs=1000,
                                                                                                                                          image_included=False,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100)

14388 6166
In epoch 0, loss: 6340.31298828125
Epoch 0: Best model saved min loss 6340.31298828125
Epoch 1: Best model saved min loss 5934.5390625
Epoch 2: Best model saved min loss 3804.020751953125
Epoch 4: Best model saved min loss 2490.83447265625
In epoch 5, loss: 1638.9195556640625
Epoch 5: Best model saved min loss 1638.9195556640625
Epoch 8: Best model saved min loss 1068.1566162109375
In epoch 10, loss: 1489.905517578125
Epoch 12: Best model saved min loss 796.4181518554688
Epoch 13: Best model saved min loss 389.5713806152344
In epoch 15, loss: 378.2738037109375
Epoch 15: Best model saved min loss 378.2738037109375
Epoch 16: Best model saved min loss 267.88397216796875
Epoch 17: Best model saved min loss 204.0205535888672
Epoch 18: Best model saved min loss 144.7747039794922
In epoch 20, loss: 212.8731231689453
Epoch 21: Best model saved min loss 137.2342529296875
Epoch 23: Best model saved min loss 103.94027709960938
Epoch 24: Best model saved min loss 59.29706954956055
In ep

100%|██████████| 10867/10867 [00:01<00:00, 7987.47it/s]


Train recall@100 0.6413160694399664
Test recall@100 0.6659159794177769


## Only Inmage

### Umap conversion

In [8]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_image_features_umap',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=True)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Node data shape : (10867, 139)
Edges data shape : (20554, 2)
Size of Input features : 100
14388 6166
In epoch 0, loss: 362.3859558105469
Epoch 0: Best model saved min loss 362.3859558105469
Epoch 1: Best model saved min loss 49.30842590332031
Epoch 2: Best model saved min loss 2.5294275283813477
Epoch 3: Best model saved min loss 0.7108553647994995
Epoch 4: Best model saved min loss 0.6415542960166931
In epoch 5, loss: 0.6258664131164551
Epoch 5: Best model saved min loss 0.6258664131164551
Epoch 6: Best model saved min loss 0.605273425579071
Epoch 7: Best model saved min loss 0.5787621736526489
Epoch 8: Best model saved min loss 0.5468220710754395
Epoch 9: Best model saved min loss 0.5087475180625916
In epoch 10, loss: 0.4648645222187042
Epoch 10: Best model saved min loss 0.4648645222187042
Epoch 11: Best model saved min loss 0.41895592212677
Epoch 12: Best model saved min loss 0.3791041374206543
Epoch 13: Best model saved min loss 0.3518766760826111
Epoch 14: Best model saved min lo

100%|██████████| 10867/10867 [00:01<00:00, 8071.37it/s]


Train recall@100 0.667577469007049
Test recall@100 0.675622048354127


### Umap and bidirectional graph

In [10]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_only_image_features_umap_bidirectional',
                                                                                                                                          n_epochs=500,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=True,use_bidirectional=True)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Node data shape : (10867, 139)
Edges data shape : (20554, 2)
Size of Input features : 100
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 372.73919677734375
Epoch 0: Best model saved min loss 372.73919677734375
Epoch 1: Best model saved min loss 39.797733306884766
Epoch 2: Best model saved min loss 1.552703857421875
Epoch 3: Best model saved min loss 0.5510805249214172
Epoch 4: Best model saved min loss 0.48610302805900574
In epoch 5, loss: 0.4377387464046478
Epoch 5: Best model saved min loss 0.4377387464046478
Epoch 6: Best model saved min loss 0.39815646409988403
Epoch 7: Best model saved min loss 0.3846866488456726
In epoch 10, loss: 0.42446786165237427
Epoch 14: Best model saved min loss 0.3813266158103943
In epoch 15, loss: 0.3676895201206207
Epoch 15: Best model saved min loss 0.3676895201206207
Epoch 16: Best model saved min loss 0.35945412516593933
Epoch 17: Best model saved min loss 0.35641881823539734
Epoch

100%|██████████| 10867/10867 [00:01<00:00, 8201.88it/s]


Train recall@100 0.6110382950649625
Test recall@100 0.5978765462443362


## Using Image Data but without dimensionality reduction ( UMAP )

In [12]:
IMAGE_MODEL = 'inception'
temp = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_image_data',
                                                                                                                                          n_epochs=1000,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt=None,patience=100)

14388 6166
In epoch 0, loss: 135.67117309570312
Epoch 0: Best model saved min loss 135.67117309570312
Epoch 1: Best model saved min loss 64.02444458007812
Epoch 2: Best model saved min loss 7.818566799163818
Epoch 3: Best model saved min loss 4.381886005401611
In epoch 5, loss: 3.6987273693084717
Epoch 5: Best model saved min loss 3.6987273693084717
Epoch 6: Best model saved min loss 3.201510429382324
Epoch 7: Best model saved min loss 1.0914448499679565
Epoch 9: Best model saved min loss 1.0666038990020752
In epoch 10, loss: 0.7249850034713745
Epoch 10: Best model saved min loss 0.7249850034713745
Epoch 11: Best model saved min loss 0.45038852095603943
Epoch 13: Best model saved min loss 0.3838786482810974
In epoch 15, loss: 0.3950403928756714
Epoch 17: Best model saved min loss 0.38094812631607056
Epoch 18: Best model saved min loss 0.3762642443180084
Epoch 19: Best model saved min loss 0.3620353043079376
In epoch 20, loss: 0.3528943657875061
Epoch 20: Best model saved min loss 0.352

100%|██████████| 10867/10867 [00:01<00:00, 7564.94it/s]


Train recall@100 0.6648932129694571
Test recall@100 0.6740290406710369


## Using Image Data along with dimensionality reduction ( UMAP )

Best hyperparameters: {'aggregator_type': 2, 'dropout': 0.42051995958759714, 'h_feats': 16.0, 'lr': 0.022254491166903718, 'n_epochs': 30.0}

In [30]:
# IMAGE_MODEL = 'inception'
# model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'mean', 'dropout': 0, 'h_feats': 16, 'lr': 0.001},
#                                                                                                                                           run_name=f'{IMAGE_MODEL}_image_data_umap',
#                                                                                                                                           n_epochs=10,
#                                                                                                                                           image_included=True,
#                                                                                                                                           split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100)

In [31]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_best_tuned_model',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':100,'n_neighbors':50,'min_dist':0.3},patience=100)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


14388 6166
In epoch 0, loss: 2402.720458984375
Epoch 0: Best model saved min loss 2402.720458984375
Epoch 1: Best model saved min loss 1656.411865234375
Epoch 2: Best model saved min loss 345.5577392578125
Epoch 3: Best model saved min loss 237.3142547607422
In epoch 5, loss: 104.38713073730469
Epoch 5: Best model saved min loss 104.38713073730469
Epoch 6: Best model saved min loss 45.30768585205078
Epoch 8: Best model saved min loss 20.57689094543457
Epoch 9: Best model saved min loss 8.738787651062012
In epoch 10, loss: 31.929183959960938
Epoch 11: Best model saved min loss 2.333468437194824
Epoch 14: Best model saved min loss 0.9150866270065308
In epoch 15, loss: 2.307528018951416
Epoch 17: Best model saved min loss 0.525970995426178
Epoch 18: Best model saved min loss 0.46912452578544617
Epoch 19: Best model saved min loss 0.4468097686767578
In epoch 20, loss: 0.40396901965141296
Epoch 20: Best model saved min loss 0.40396901965141296
Epoch 21: Best model saved min loss 0.387753844

100%|██████████| 10867/10867 [00:01<00:00, 7722.31it/s]


Train recall@100 0.667577469007049
Test recall@100 0.675622048354127


In [12]:
IMAGE_MODEL = 'inception'
model,g,nodes,edges,train_pos_u, train_pos_v,test_pos_u, test_pos_v,train_neg_u, train_neg_v,test_neg_u, test_neg_v,train_g,test_g = train_model(mlflow=mlflow,image_model=IMAGE_MODEL,model_params = {'aggregator_type': 'lstm', 'dropout': 0.42051995958759714, 'h_feats': 16, 'lr': 0.022254491166903718},
                                                                                                                                          run_name=f'{IMAGE_MODEL}_umap_bidirectional',
                                                                                                                                          n_epochs=100,
                                                                                                                                          image_included=True,
                                                                                                                                          split_ratio=0.3,umap_dt={'n_components':1,'n_neighbors':50,'min_dist':0.3},patience=100,only_image=True,use_bidirectional=True)

c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\umap\umap_.py:1945: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")
c:\Users\Kartik\anaconda3\envs\tf\lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(
2024/08/18 08:48:14 WARNING mlflow.sklearn: Training metrics will not be recorded because training labels were not specified. To automatically record training metrics, provide training labels as inputs to the model training function.
2024/08/18 08:48:14 WARNING mlflow.sklearn: Failed to infer model signature: the trained model does not have a `predict` or `transform` function, which is required in order to infer the signature
2024/08/18 08:48:14 WARNING mlflow.sklearn: Model was missing function: predict. Not loggi

Node data shape : (10867, 40)
Edges data shape : (20554, 2)
Size of Input features : 38
##########################
Converting to biderctional Graph
##########################
27998 11999
In epoch 0, loss: 6671.4052734375
Epoch 0: Best model saved min loss 6671.4052734375
Epoch 1: Best model saved min loss 5143.75146484375
Epoch 3: Best model saved min loss 2271.85693359375
In epoch 5, loss: 1742.8062744140625
Epoch 5: Best model saved min loss 1742.8062744140625
Epoch 7: Best model saved min loss 1108.091552734375
Epoch 8: Best model saved min loss 667.4564208984375
In epoch 10, loss: 430.9026794433594
Epoch 10: Best model saved min loss 430.9026794433594
Epoch 13: Best model saved min loss 264.24749755859375
In epoch 15, loss: 231.84072875976562
Epoch 15: Best model saved min loss 231.84072875976562
Epoch 17: Best model saved min loss 131.11282348632812
Epoch 19: Best model saved min loss 102.09928894042969
In epoch 20, loss: 56.513126373291016
Epoch 20: Best model saved min loss 56.5

100%|██████████| 10867/10867 [00:01<00:00, 8076.73it/s]


Train recall@100 0.608480661426549
Test recall@100 0.5866506245706534


KeyboardInterrupt: 